<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Time Series Properties, Autoregressive, and Moving Average Models

_Authors: Kiefer Katovich (SF), Justin Pounders_

---
<a id="learning-objectives"></a>
<a id="learning-objectives"></a>

### Learning Objectives

- Describe and calculate a time series decomposition.
- Calculate and plot ACF and PACF
- Create autoregressive and moving average models


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

sns.set_style('whitegrid')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

<a id="load-the-unemployment-data"></a>
<a id="load-the-unemployment-data"></a>
## Load the Unemployment Data
---

This is historical quarterly data regarding unemployment rates in the United States. First, perform any necessary cleaning.

In [ ]:
data = pd.read_csv('./datasets/seasonally-adjusted-quarterly-us.csv')

In [ ]:
data.columns = ['year_quarter', 'unemployment_rate']
data['unemployment_rate'] = data['unemployment_rate'].map(lambda x: float(str(x).replace('%','')))
data.dropna(inplace=True)

<a id="create-a-datetime-index"></a>
<a id="create-a-datetime-index"></a>
## Create a Datetime Index
---

This is quarterly data, so converting to datetime is a bit complicated. `.dt.to_period('Q')` will help us represent the string as a datetime object.

<a id="visually-examine-the-unemployment-rate"></a>
<a id="visually-examine-the-unemployment-rate"></a>
## Visually Examine the Unemployment Rate
---

Plot the unemployment rate.

In [ ]:
data['unemployment_rate'].plot(lw=2.5, figsize=(12,5))

In [ ]:
date_ticks = data.index.to_timestamp()

<a id="autocorrelation-and-the-autocorrelation-function-acf"></a>
<a id="autocorrelation-and-the-autocorrelation-function-acf"></a>
##  Autocorrelation and the Autocorrelation Function (ACF)
---

In previous weeks, our analyses have been concerned with the correlation between two or more variables (height and weight, education and salary, etc.). In time series data, autocorrelation is a measure of _how correlated a variable is with itself_.

Specifically, autocorrelation measures how closely earlier values relate to values that occur later in time.

Examples of autocorrelation are:

    In stock market data, the stock price at one point is correlated with the stock 
    price of the point that's directly prior in time. 
    
    In sales data, sales on a Saturday are likely correlated with 
    sales on the next Saturday and the previous Saturday, as well as other days to more
    or less of an extent.

Below is the formula for the autocorrelation function (ACF):

$\text{Given measurements } x_1, x_2, x_3 ... x_n \text{ at time points } t_1, t_2, t_3 ... t_n:$

### $$lag_k\;acf() = \frac{\sum_{t=k+1}^{n}\left(\;x_t - \bar{x}\;\right)\left(\;x_{t-k} - \bar{x}\;\right)}{\sum_{t=1}^n\left(\;x_t - \bar{x}\;\right)^2}$$

Compare this to the formula for correlation:

$\text{Given measurements } x_1, x_2, x_3 ... x_n \text{ and measurements } y_1, y_2, y_3 ... y_n:$

### $$r_{xy} = \frac{\sum_{i=1}^{n}\left(\;x_i - \bar{x}\;\right)\left(\;y_{i} - \bar{y}\;\right)}{\sqrt{\left(\sum_{i=1}^{n}\left(\;x_i - \bar{x}\;\right)^2\sum_{i=1}^n\left(\;y_i - \bar{y}\;\right)^2\right)}}$$


<a id="computing-autocorrelation"></a>
<a id="computing-autocorrelation"></a>
### Computing Autocorrelation

To compute autocorrelation, we fix a lag, _k_, which is the delta between the given point and the prior point used to compute the correlation.

With a _k_ value of 1, we'd compute how correlated a value is with the value that's immediately prior. With a _k_ value of 10, we'd compute how correlated a variable is with the value that's 10 time points prior.

<a id="autocorrelation-using-statsmodels"></a>
<a id="autocorrelation-using-statsmodels"></a>
### Autocorrelation Using Statsmodels

Statsmodels comes with some convenient packages for calculating and plotting autocorrelation. Load up these two functions and try them out:

```python
from statsmodels.tsa.stattools import acf
from statsmodels.graphics.tsaplots import plot_acf
```

(C1) **Check:** 

1. What does autocorrelation tell you?
2. What is a shortcoming of autocorrelation?

<a id="partial-autocorrelation-and-the-partial-autocorrelation-function-pacf"></a>
<a id="partial-autocorrelation-and-the-partial-autocorrelation-function-pacf"></a>
## Partial Autocorrelation and the Partial Autocorrelation Function (PACF)
---

Another important chart for diagnosing your time series is the partial autocorrelation chart (PACF). This is similar to autocorrelation, but, instead of being just the correlation at increasing lags, it is the correlation at a given lag, _controlling for the effect of previous lags._

Load up the sister functions for partial autocorrelation from Statsmodels and test them out on the differenced time series.

```python
from statsmodels.tsa.stattools import pacf
from statsmodels.graphics.tsaplots import plot_pacf
```

--- 

## Time Series Decomposition

$$ Y_t = T_t + S_t + C_t + \varepsilon_t$$

- $Y_t = $ observed value at time $t$
- $T_t = $ trend component, *long-run behavior*
- $S_t = $ seasonality component, *periodic fluctuations*
- $C_t = $ cyclical component, *non-periodic fluctuations*
- $\varepsilon_t = $ noise component, *we would like this to be stationary*

> Decomposition above is additive; can also be multiplicative.


There are several algorithms for performing this decomposition

- Classical decomp., [https://www.otexts.org/fpp/6/3](https://www.otexts.org/fpp/6/3)
- X-12-ARIMA, [https://www.otexts.org/fpp/6/4](https://www.otexts.org/fpp/6/4)
- STL, [https://www.otexts.org/fpp/6/5](https://www.otexts.org/fpp/6/5)

See also `seasonal_decompose` in `statsmodel.tsa.seasonal`.  

> Nice tutorial here: [http://machinelearningmastery.com/decompose-time-series-data-trend-seasonality/](http://machinelearningmastery.com/decompose-time-series-data-trend-seasonality/)


In [ ]:
# Make some fake data
n = 201
np.random.seed(42)
t = np.linspace(0, 500, n)
signal = (t/250)**2 + np.sin(t/10) + np.random.normal(scale=0.15, size=n)

d0 = datetime.datetime.now()
dt1 = datetime.timedelta(1)
signal = pd.Series(signal, index=[d0+dt1*i for i in t])

signal.plot()

(C2) **Is the above *stationary***?

**Questions:** 
- *What is the long-term behavior of my series?*
- *How does my time series fluctuate?*


In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

result = seasonal_decompose(signal)
result.plot();

Hmmm... definitely some seasonality still left in the residual. From the `statsmodels` docs:

> "This is a naive decomposition. More sophisticated methods should be preferred."

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

tsindex = data.index.to_timestamp()
data_ts = data.copy()
data_ts.index = tsindex

result = seasonal_decompose(data_ts.unemployment_rate)
result.plot();

In [ ]:
result.resid[:10]

(C2) **Check**

Which of the three trends addresses the following questions?
1. What is the long-term behavior of my series?
2. What is the effect of time-of-day on my series?
3. What is the effect of larger, unseen fluctuations on my series?


<a id="differencing-a-timeseries-and-stationarity"></a>
<a id="differencing-a-timeseries-and-stationarity"></a>
## "Differencing" a Time Series and Stationarity
---

If a time series is stationary, the mean, variance, and autocorrelation (covered in the next section) will be constant over time. 

> Forecasting methods typically assume the time series you are forecasting on to be stationary — or at least approximately stationary.

To stationarize (maybe not a word...) we could possibly take two approaches:

1. Pull the "residual" component from the decomposition and check if it is stationary, or
2. Take a "diff" one or more times.

The most common way to make a time series stationary is through "differencing." This procedure converts a time series into the difference between values.

<a id="-delta-yt--yt---yt--"></a>
### $$ \Delta y_t = y_t - y_{t-1} $$

This removes trends in the time series and ensures that the mean across time is zero. In most cases, this only requires a single difference, although, in some cases, a second difference (or third, etc.) will be necessary to remove trends.

**Difference the unemployment rate and plot.**

In [ ]:
# A:

> How do you really know if your timeseries is stationary?  You can formulate stationarity as a hypothesis and then test the hypothesis!  An example of this approach is the [Dickey-Fuller test](https://en.wikipedia.org/wiki/Dickey%E2%80%93Fuller_test).  We'll look at this in the afternoon.

<a id="autoregressive-ar-models"></a>
<a id="autoregressive-ar-models"></a>
## Autoregressive (AR) Models

---

Autoregressive (AR) models use data from previous time points to predict the next time point. These are essentially regression models in which the predictors are previous time points of the outcome.

Typically, AR models are denoted as `AR(p)`, where _p_ indicates the number of previous time points to incorporate. `AR(1)` is the most common.

In an autoregressive model, we learn regression coefficients on the features that are the previous _p_ values.

### $$y_i = c + \beta_1  y_{i-1} + \beta_2  y_{i-2}\ +\ ...\ +\ \beta_p  y_{i-p}\ +\ \epsilon \\
y_i =\sum_{j=1}^p \beta_j y_{i-j} + \epsilon$$

We can build autoregressive models using the `ARMA` class from Statsmodels. (Alternatively, there is a newer Python package, PyFlux, that also looks promising for time series modeling.)

In [ ]:
from statsmodels.tsa.arima_model import ARMA
import pyflux as pf

In [ ]:
# A:

### "In-sample" predictions

In [ ]:
predictions = None
full_pred = None

fig, ax = plt.subplots(figsize=(12,5))
ax.plot(date_ticks[1:], data.unemployment_rate[1:], lw=2, color='darkred', label='Data')
ax.plot(date_ticks[1:], full_pred, lw=2, color='grey', ls='dashed', label='Fit')
plt.legend()

fig, ax = plt.subplots(figsize=(12,5))
ax.plot(date_ticks[1:], udiff, lw=2, color='grey', ls='dashed',label='Data')
ax.plot(date_ticks[1:], predictions, lw=2, color='darkred',label='Fit')
plt.legend()
plt.show()

### "Out-of-sample" predictions

In [ ]:
predictions_oos = None

fig, ax = plt.subplots(figsize=(12,5))
ax.plot(date_ticks[1:], udiff, lw=2, color='grey', ls='dashed', label='Data')
ax.plot(date_ticks[1:], predictions, lw=2, color='darkred', label='In-sample predictions')
ax.plot(date_ticks[101:], predictions_oos, lw=2, color='blue', label='Out-of-sample predictions')
plt.legend()
plt.show()

(C3) **Check**

What's the difference between in-sample and out-of-sample prediction?

<a id="moving-average-ma-models"></a>
<a id="moving-average-ma-models"></a>
## Moving Average (MA) Models
---

**Moving average models** take previous _error terms_ as inputs. They predict the next value based on deviations from previous predictions. _This can be useful for modeling a sudden occurrence — e.g., something going out of stock affecting sales or a sudden rise in popularity._

As with autoregressive models, we have an order term, _q_, and we refer to our model as `MA(q)`. This moving average model is dependent on the last _q_ errors. If we have a time series of sales per week, $y_i$, we can regress each $y_i$ on the last _q_ error terms.

### $$y_t = \epsilon_t + \beta_{1} \epsilon_{t-1} + ... \beta_{n} \epsilon_{t-n} \\
y_t = \sum_{i=1}^n \beta_i \epsilon_{t-i} + \epsilon_t$$

Sometimes, the mean of the time series is included in the equation:

### $$ y_t = \mu + \sum_{i=1}^n \beta_i \epsilon_{t-i} + \epsilon_t $$

Moving average models are not as trivial to fit as autoregressive models because the error terms are unobserved. There are a variety of different ways you can estimate the parameters, some of which are covered in [this paper](https://www.it.uu.se/research/publications/reports/2006-022/2006-022-nc.pdf).

With the simpler fitting procedures, a model is iteratively fit, then errors are computed and refit over and over again until the parameters on the errors converge.

MA includes the mean of the time series. The behavior of the model is therefore characterized by random jumps around the mean value.

In an `MA(1)` model, one coefficient on the error of our previous prediction affects our estimate for the next value in the time series.


In [ ]:
# A:

In [ ]:
fig, ax = plt.subplots(figsize=(12,5))
ax.plot(date_ticks[1:], udiff, lw=2, color='grey', ls='dashed',label='Data')
ax.plot(date_ticks[1:], ma1.fittedvalues, lw=2, color='darkred',label='Fit')
plt.legend()
plt.show()

<a id="arma-and-arima-models"></a>
<a id="arma-and-arima-models"></a>
## ARMA and ARIMA Models
---

**ARMA** models combine autoregressive models and moving average models, parameterizing the behavior of models with `p` and `q` terms that correspond to `AR(p)` models and `MA(q)` models.

Autoregressive models slowly incorporate changes in preferences, tastes, and patterns. Moving average models base their predictions not on the prior value but instead on the prior error, allowing us to correct sudden changes based on random events — supply, popularity spikes, etc.

**ARIMA** is just like the `ARMA(p, q)` model, but, rather than predicting the value of the series, it predicts the _differenced_ series or changes in the series. The order of differencing is set by a _d_ term — as in `ARIMA(p, d, q)` — or alternatively, you can just fit an `ARMA(p, q)` model on a differenced time series.

Recall the Pandas `diff()` function. This computes the difference between two consecutive values. In an ARIMA model, we attempt to predict this difference instead of the actual values.

### $$y_t - y_{(t-1)} = ARMA(p, q)$$

In modeling, we assume time series to be "stationary." This handles the stationarity assumption: Instead of detrending or differencing manually, models do this via differencing terms.

In [ ]:
# A:

In [ ]:
fig, ax = plt.subplots(figsize=(12,5))
ax.plot(date_ticks[1:], udiff, lw=2, color='grey', ls='dashed',label='Data')
ax.plot(date_ticks[1:], ar1ma1.fittedvalues, lw=2, color='darkred',label='Fit')
plt.legend()
plt.show()

In [ ]:
full_pred = data.unemployment_rate.values[0]+np.cumsum(ar1ma1.fittedvalues)

fig, ax = plt.subplots(figsize=(12,5))
ax.plot(date_ticks[1:], full_pred, lw=2, color='grey', ls='dashed')
ax.plot(date_ticks[1:], data.unemployment_rate[1:], lw=2, color='darkred')
plt.show()

<a id="how-to-choose-the-right-p-and-q-parameters"></a>
<a id="how-to-choose-the-right-p-and-q-parameters"></a>
## How to Choose the Right `p` and `q` Parameters
---

In general, it's never a bad idea to choose your parameters based on hold-out testing; that is to say, checking the performance of your model on future time points based on different choices of `p` and `q` for an ARIMA model.

However, you can get a sense for which parameters will work best based on autocorrelation and partial autocorrelation charts.

[This site](https://people.duke.edu/~rnau/411arim3.htm) offers a detailed overview of how to use ACF and PACF to determine your parameters.

Below are some basic general guidelines. Remember that these rules apply to the ACF and PACF plots of differenced time series rather than the original time series (the exception being if your time series is stationary and does not require differencing):

1. If the PACF has a sharp cut-off and the lag-1 ACF value is positive, choose an `AR(x)` term where `x` is the lag in the PACF after the cut-off.
2. If the ACF has a sharp cut-off and the lag-1 ACF value is negative, choose an `MA(x)` term where `x` is the lag in the ACF after the cut-off.
3. If both the ACF and PACF show a gradual decay, an ARMA model is likely appropriate as opposed to AR or MA alone.

Context 1 above corresponds to time series that are "underdifferenced," as indicated by a positive autocorrelation at lat 1. Likewise, Context 2 is "overdifferenced," as indicated by a negative autocorrelation.

In general, you should try to choose an AR or MA model as opposed to an ARMA model. The AR and MA terms can work against each other in the model and create an overly complex representation.